# NSO Restconf - DNS Example

This notebook provides an example of how to interact with NSO programmatically using `REST` APIs.

The `REST` API are implemented in the `dns_restconf.py` module. 

The module provides several methods to list devices, synchronize device configurations, and perform a dry-run addition of a DNS server.

## Lab Setup

By following these steps, you will have the lab environment set up and ready to proceed with the experiment.

The [Makefile](Makefile) checks all the environment variables are set, any env var missing will cause the `Makefile` to fail.

In [ ]:
%env NCS_RUN_DIR=~/nso-lab-rundir
%env REPO_DIR=~/src/nso-restconf-dns-example

Source the `ncsrc` file.

In [ ]:
! source $NCS_DIR/ncsrc

Build the environment. Everytime you build the environment, netsim & ncs are stopped, then the environment on `$NCS_RUN_DIR` is deleted. This is useful to start from scratch everytime when experimenting.

In [ ]:
! make -C $REPO_DIR build

---

## Initialization

First, initialize the session handler with your NSO credentials and base URL.


In [ ]:
from dns_restconf import DnsHandler

USERNAME = "admin"
PASSWORD = "admin"
BASE_URL = "http://localhost:8080/restconf"

restconf = DnsHandler(USERNAME, PASSWORD, BASE_URL)

---

#### List Devices in NSO
To list all devices in the NSO, we created the `list_devices_in_nso` method. This method sends a GET request to the NSO and prints the results.

- REST Path: `/data?fields=tailf-ncs:devices/device(name;address)`
- REST Operation: `GET`

In [ ]:
restconf.list_devices_in_nso()

---

#### Synchronize Configuration from a Single Device
To synchronize the configuration from a single device, the `nso_sync_from_single_device` method was created. This method sends a `POST` request to the NSO for the specified device and prints the results.

- REST Path: `/operations/tailf-ncs:devices/device={device}/sync-from`
- REST Operation: `POST`

In [ ]:
restconf.nso_sync_from_single_device(device="ex1")

---

#### Synchronize Configuration from All Devices
To synchronize the configuration from all devices, we added the `nso_sync_from` function. This method sends a `POST` request to the NSO and prints the results.

- REST Path: `/operations/tailf-ncs:devices/sync-from`
- REST Operation: `POST`

In [ ]:
restconf.nso_sync_from()

---

#### Add DNS Server
To add a DNS server to a device, we are using the `add_dns_server` method. This method sends a `PATCH` request to the NSO with the specified device and DNS server details and prints the results.

- REST Path: `/data`
- REST Operation: `PATCH`
- Payloud:

```json
{
  "data": {
    "dns-config:dns-config": [
      {"device": device, "dns-server": [{"address": dns_server}]}
    ]
  }
}
```
> Payload is based on the yang model of the service

In [ ]:
restconf.add_dns_server(device="ex1", dns_server="1.1.1.1")

---

#### Check DNS Configuration
To check the DNS configuration of a device, we use the `check_dns_config` method. This method sends a `GET` request to the NSO and prints the results.

- REST Path: `/data/tailf-ncs:devices/device={device}/config/tailf-ned-cisco-ios:ip/name-server/`
- REST Operation: `GET`

In [ ]:
restconf.check_dns_config(device="ex1")

---

#### Dry-Run Add DNS Server
To perform a dry-run addition of a DNS server to a device, use the `dry_run_add_dns_server` method. This method sends a `PATCH` request to the NSO with the specified device and DNS server details and prints the results.

- REST Path: `/data?dry-run=native`
- REST Operation: `PATCH`
- Payload:

```json
{
  "data": {
    "dns-config:dns-config": [
      {"device": device, "dns-server": [{"address": dns_server}]}
    ]
  }
}
```
> Payload is based on the yang model of the service

In [ ]:
restconf.dry_run_add_dns_server(device="ex1", dns_server="2.2.2.2")

---

Review existing configuration on `ex1`, confirm no changes were pushed.
- REST Path: `/data/tailf-ncs:devices/device={device}/config/tailf-ned-cisco-ios:ip/name-server/`
- REST Operation: `GET`

In [ ]:
restconf.check_dns_config(device="ex1")

---

Add a DNS server to `ex1`.
- REST Path: `/data`
- REST Operation: `PATCH`
- Payloud:

```json
{
  "data": {
    "dns-config:dns-config": [
      {"device": device, "dns-server": [{"address": dns_server}]}
    ]
  }
}
```

In [ ]:
restconf.add_dns_server(device="ex1", dns_server="2.2.2.2")

---

Confirm new DNS server is present in `ex1`.
- REST Path: `/data/tailf-ncs:devices/device={device}/config/tailf-ned-cisco-ios:ip/name-server/`
- REST Operation: `GET`

In [ ]:
restconf.check_dns_config(device="ex1")

---

#### List Rollback Files
To list all rollback files, use the `list_rollback_files` method. This method sends a `GET` request to the NSO and prints the results.

- REST Path: `/data/tailf-rollback:rollback-files`
- REST Operation: `GET`

In [ ]:
restconf.list_rollback_files()

---

#### Apply Rollback File
To apply a rollback file, use the `apply_rollback_file` method. This method sends a `POST` request to the NSO with the specified rollback ID and prints the results.

- REST Path: `/data/tailf-rollback:rollback-files/apply-rollback-file`
- REST Operation: `POST`

In [ ]:
restconf.apply_rollback_file(rollback_id=0)

---

See the changes done on `ex1` after the rollback.
- REST Path: `/data/tailf-ncs:devices/device={device}/config/tailf-ned-cisco-ios:ip/name-server/`
- REST Operation: `GET`

In [ ]:
restconf.check_dns_config(device="ex1")

---

Dry-run adding a new DNS server.
- REST Path: `/data?dry-run=native`
- REST Operation: `PATCH`
- Payload:

```json
{
  "data": {
    "dns-config:dns-config": [
      {"device": device, "dns-server": [{"address": dns_server}]}
    ]
  }
}
```
> Payload is based on the yang model of the service

In [ ]:
restconf.dry_run_add_dns_server(device="ex1", dns_server="3.3.3.3")

---

Review existing configuration on `ex1`, confirm no changes were pushed.
- REST Path: `/data/tailf-ncs:devices/device={device}/config/tailf-ned-cisco-ios:ip/name-server/`
- REST Operation: `GET`

In [ ]:
restconf.check_dns_config(device="ex1")

---

Add a DNS server to `ex1`.
- REST Path: `/data`
- REST Operation: `PATCH`
- Payloud:

```json
{
  "data": {
    "dns-config:dns-config": [
      {"device": device, "dns-server": [{"address": dns_server}]}
    ]
  }
}
```
> Payload is based on the yang model of the service

In [ ]:
restconf.add_dns_server(device="ex1", dns_server="3.3.3.3")

---

Confirm new DNS server is present in `ex1`.
- REST Path: `/data/tailf-ncs:devices/device={device}/config/tailf-ned-cisco-ios:ip/name-server/`
- REST Operation: `GET`

In [ ]:
restconf.check_dns_config(device="ex1")

---

### Clean Up

If needed you can clean up the exercise to practice more.

In [ ]:
! make -C $REPO_DIR clean

---

### More Restconf Examples

To find more restconf examples that come with NSO do

In [ ]:
! find $NCS_DIR/examples.ncs/ -type f -name "showcase_rc.py"